In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import typing as tp

from tqdm import tqdm
from collections import Counter
from sklearn.model_selection import train_test_split
from functools import partial

sns.set_theme()

In [14]:
import pip
pip.main(['install', 'tmdbsimple'])
import requests
import time
import tmdbsimple as tmdb


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


13:22:38 start blocking IO
13:22:38 finished blocking IO


{'adult': False,
 'backdrop_path': '/yB2hTgz9CTVYjlMWPSl3LPx5nWj.jpg',
 'genre_ids': [35, 18, 53],
 'id': 466272,
 'original_language': 'en',
 'original_title': 'Once Upon a Time… in Hollywood',
 'overview': 'Los Angeles, 1969. TV star Rick Dalton, a struggling actor specializing in westerns, and stuntman Cliff Booth, his best friend, try to survive in a constantly changing movie industry. Dalton is the neighbor of the young and promising actress and model Sharon Tate, who has just married the prestigious Polish director Roman Polanski…',
 'popularity': 39.733,
 'poster_path': '/8j58iEBw9pOXFD2L0nt0ZXeHviB.jpg',
 'release_date': '2019-07-24',
 'title': 'Once Upon a Time… in Hollywood',
 'video': False,
 'vote_average': 7.5,
 'vote_count': 8618}

In [2]:
!ls

password-complexity.ipynb


In [3]:
train = pd.read_csv('../datasets/password-complexity/train.csv')

In [4]:
test = pd.read_csv('../datasets/password-complexity/test.csv')

In [5]:
train = train.fillna("")
test = test.fillna("")

In [6]:
train.loc[train.Times.sort_values(ascending=False)[:50].index]

,Password,Times
2715397,123456,55893
3136279,qwerty,13137
1175081,123456789,11696
2363307,12345,10938
2988373,1234,6432
2307329,111111,5682
3348280,1234567,4796
1795496,dragon,3927
3336606,123123,3845
3798071,baseball,3565


In [7]:
train = train[train.Password.apply(lambda x: isinstance(x, str))]

In [8]:
# from collections import defaultdict

# bi = defaultdict(int)

# for p in tqdm(train.Password):
#     for l, r in zip(p, p[1:]):
#         bi[l+r] += 1
# bi = Counter(bi)
# # bi.most_common(100)

# tri = defaultdict(int)

# for p in tqdm(train.Password):
#     for l, m, r in zip(p, p[1:], p[2:]):
#         tri[l+m+r] += 1
# tri = Counter(tri)
# # tri.most_common(200)

# four = defaultdict(int)

# for p in tqdm(train.Password):
#     for l, m, c, r in zip(p, p[1:], p[2:], p[3:]):
#         four[l+m+c+r] += 1
# four = Counter(four)

In [1]:
import re

def hasNumbers(inputString):
    return bool(re.search(r'\d', inputString))

def hasLetters(inputString):
    return bool(re.search(r'\w', inputString))

def number_of_characters(pswrd):
    return len(pswrd) * 4

def uppercase_letters(pswrd):
    return (len(pswrd) - sum(1 for c in pswrd if c.isupper())) * 2

def lowercase_letters(pswrd):
    return (len(pswrd) - sum(1 for c in pswrd if c.islower())) * 2

def number_of_numbers(pswrd):
    return len(re.findall(r'\d', pswrd)) * 4

def number_of_symbols(pswrd):
    return (len(pswrd) - len(re.findall(r'\w', pswrd))) * 6 

def middel_numbers_or_symbols(pswrd):
    if len(pswrd) > 2:
        pswrd = pswrd[1:-1] 
        return ((len(pswrd) - len(re.findall(r'\w', pswrd))) + len(re.findall(r'\d', pswrd))) * 2 
    else:
        return 0

def letters_only(pswrd):
    return len(pswrd) * pswrd.isalpha()

def numbers_only(pswrd):
    return len(pswrd) * pswrd.isdigit()

def consecutive_uppercase_letters(pswrd):
    score = 0
    for l, r in zip(pswrd, pswrd[1:]):
        score += l.isupper() and r.isupper()
    return score * 2

def consecutive_lowercase_letters(pswrd):
    score = 0
    for l, r in zip(pswrd, pswrd[1:]):
        score += l.islower() and r.islower()
    return score * 2

def consecutive_numbers(pswrd):
    score = 0
    for l, r in zip(pswrd, pswrd[1:]):
        score += l.isdigit() and r.isdigit()
    return score * 2

def has_top_substr(pswrd, s):
    return pswrd.count(s)

feature_funcs = [
    number_of_characters,
    uppercase_letters,
    lowercase_letters,
    number_of_numbers,
    number_of_symbols,
    middel_numbers_or_symbols,
    letters_only,
    numbers_only,
    consecutive_uppercase_letters,
    consecutive_lowercase_letters,
    consecutive_numbers
]

# for f in tqdm(feature_funcs):
#     train[f.__qualname__] = train.Password.apply(f).astype('int8', copy=False)

# for b in tqdm(map(lambda x: x[0], bi.most_common(200))):
#     X_te[b] = X_te.Password.apply(has_top_substr, s=b).astype('int8', copy=False)

# for t in tqdm(map(lambda x: x[0], tri.most_common(150))):
#     X_te[t] = train.Password.apply(has_top_substr, s=t).astype('int8', copy=False)
    
# for f in tqdm(map(lambda x: x[0], four.most_common(100))):
#     train[f] = train.Password.apply(has_top_substr, s=f).astype('int8', copy=False)
# hasLetters("631XniVx2lS5I")
# train['has_numbers'] = train.Password.apply(hasNumbers)
# train['has_letters'] = train.Password.apply(hasLetters)

In [10]:
base_features = {
    "Password": str,
    "Times": np.int32,
    "number_of_characters": np.int8,
    "uppercase_letters": np.int8,
    "lowercase_letters": np.int8,
    "number_of_numbers": np.int8,
    "number_of_symbols": np.int8,
    "middel_numbers_or_symbols": np.int8,
    "letters_only": np.int8,
    "numbers_only": np.int8,
    "consecutive_uppercase_letters": np.int8,
    "consecutive_lowercase_letters": np.int8,
    "consecutive_numbers": np.int8,
}

In [11]:
train = train[list(base_features.keys())]

In [12]:
# fit_part, test_part = train_test_split(train, train_size=0.9, random_state=17)

# train_part, val_part = train_test_split(fit_part, train_size=0.7, random_state=17)

# X_tr, y_tr = train_part.drop(columns=['Times', 'Password']), train_part.Times 
# X_va, y_va = val_part.drop(columns=['Times', 'Password']), val_part.Times 
# X_te, y_te = test_part.drop(columns=['Times', 'Password']), test_part.Times 

# del train_part, val_part, test_part 

In [13]:
# import catboost as cb
from sklearn.metrics import mean_squared_log_error as msle

def rmsle(y_true, y_pred):
    return msle(y_true, y_pred) ** 0.5

# train_pool = cb.Pool(X_tr, label=y_tr)
# val_pool = cb.Pool(X_va, label=y_va)

# cb_m = cb.CatBoostRegressor(loss_function='RMSE', iterations=500)
# cb_m.fit(train_pool, eval_set=val_pool)

# y_pred = cb_m.predict(X_va)

# y_pred = np.maximum(0, y_pred)
# print("max 0 and pred: ", rmsle(y_pred, y_va))
# y_pred = np.maximum(1, y_pred)
# print("max 1 and pred: ", rmsle(y_pred, y_va))
# y_pred = y_pred / y_pred.mean() * y_tr.mean()
# print("max 1 and pred and normed: ", rmsle(y_pred, y_va))

# plt.hist(y_pred, bins=np.arange(-7, 10, 0.1));
# print(y_pred.mean())
# print(rmsle([1.3] * len(y_va), y_va))

## xgboost

In [17]:
import pip
pip.main(['install', 'xgboost'])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


0

In [14]:
import xgboost as xgb

In [15]:
train

,Password,Times,number_of_characters,uppercase_letters,lowercase_letters,number_of_numbers,number_of_symbols,middel_numbers_or_symbols,letters_only,numbers_only,consecutive_uppercase_letters,consecutive_lowercase_letters,consecutive_numbers
0,631XniVx2lS5I,2,52,18,18,20,0,8,0,0,0,2,4
1,LEGIT747,1,32,6,16,12,0,4,0,0,8,0,4
2,742364es,1,32,16,12,24,0,10,0,0,0,2,10
3,3846696477,1,40,20,20,40,0,16,0,10,0,0,18
4,laurahop,2,32,16,0,0,0,0,8,0,0,14,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4151491,7503ri,1,24,12,8,16,0,6,0,0,0,2,6
4151492,trou,2,16,8,0,0,0,0,4,0,0,6,0
4151493,300762,7,24,12,12,24,0,8,0,6,0,0,10
4151494,seaurchin007,1,48,24,6,12,0,4,0,0,0,16,4


In [27]:
# dtrain = xgb.DMatrix(X_tr, label=y_tr)
# dval = xgb.DMatrix(X_va, label=y_va)
# dtest = xgb.DMatrix(X_te, label=y_te)

# # del X_tr

In [44]:
fit_part, test_part = train_test_split(train, train_size=0.9, random_state=17)

X_tr, y_tr = fit_part.drop(columns=['Times', 'Password']), fit_part.Times 
X_te, y_te = test_part.drop(columns=['Times', 'Password']), test_part.Times 

param = {
    'objective': 'reg:squaredlogerror',
    'nthread': -1,
    'eval_metric': 'rmsle',
    'n_estimators': 100,
}

evallist = [(X_te, y_te)]
model = xgb.XGBRegressor(**param)
model.fit(X_tr, y_tr, eval_set=evallist, early_stopping_rounds=30)

[0]	validation_0-rmsle:0.52446
[1]	validation_0-rmsle:0.47999
[2]	validation_0-rmsle:0.44801
[3]	validation_0-rmsle:0.42617
[4]	validation_0-rmsle:0.41199
[5]	validation_0-rmsle:0.40318
[6]	validation_0-rmsle:0.39789
[7]	validation_0-rmsle:0.39482
[8]	validation_0-rmsle:0.39307
[9]	validation_0-rmsle:0.39199
[10]	validation_0-rmsle:0.39147
[11]	validation_0-rmsle:0.39108
[12]	validation_0-rmsle:0.39089
[13]	validation_0-rmsle:0.39077
[14]	validation_0-rmsle:0.39067
[15]	validation_0-rmsle:0.39063
[16]	validation_0-rmsle:0.39063
[17]	validation_0-rmsle:0.39059
[18]	validation_0-rmsle:0.39058
[19]	validation_0-rmsle:0.39055
[20]	validation_0-rmsle:0.39054
[21]	validation_0-rmsle:0.39051
[22]	validation_0-rmsle:0.39052
[23]	validation_0-rmsle:0.39052
[24]	validation_0-rmsle:0.39051
[25]	validation_0-rmsle:0.39051
[26]	validation_0-rmsle:0.39051
[27]	validation_0-rmsle:0.39050
[28]	validation_0-rmsle:0.39049
[29]	validation_0-rmsle:0.39049
[30]	validation_0-rmsle:0.39049
[31]	validation_0-

In [21]:
# bst.save_model('../ml-models/password-complexity/xgboost_baseline.json')

# bst_ = xgb.Booster()
# bst_.load_model('../ml-models/password-complexity/xgboost_baseline.json')

In [31]:
rmsle(y_te, bst_.predict(dtest))

0.3904817199116006

In [11]:
import xgboost as xgb
import sklearn as skl
print(xgb.__version__, skl.__version__)

('1.4.1', '0.24.1')

In [4]:
import pickle
import pandas as pd

In [48]:
with open('../ml-models-service/ml_models/password_complexity/xgboost_baseline.pkl', 'wb') as file:
    pickle.dump(model, file)

In [3]:
with open('../ml-models-service/ml_models/password_complexity/xgboost_baseline.pkl', 'rb') as f:
    model = pickle.load(f)

In [5]:
def get_features(password):
    data = pd.DataFrame([password], columns=['password'])
    for f in feature_funcs:
        data[f.__qualname__] = data.password.apply(f).astype('int8', copy=False)
    return data.drop(columns=['password'])

In [6]:
model.predict(get_features('qwer'))[0]

2.1998448

In [37]:
xgb.XGBRegressor().predict([1])

NotFittedError: need to call fit or load_model beforehand

In [37]:
pd.DataFrame(['aaa'], columns=['password'])

,password
0,aaa


In [ ]:
# sub["Times"] = pd.Series(y_sub)
# sub.set_index("Id").to_csv('my_sub.csv')